In [1]:
import os,sys,datetime
import numpy as np
from scipy.spatial.distance import *
from copy import deepcopy
from hybrid_mdmc.data_file_parser import parse_data_file
from hybrid_mdmc.lammps_files_classes import write_lammps_data, write_lammps_init
from hybrid_mdmc.customargparse import HMDMC_ArgumentParser
from hybrid_mdmc.classes import *
from hybrid_mdmc.parsers import *
from hybrid_mdmc.kmc import *
from hybrid_mdmc.functions import *
from hybrid_mdmc.calc_voxels import *
from hybrid_mdmc.diffusion import *

In [47]:
data_file = 'test.end.data'
rxndf = 'test.rxndf'
msf = 'test.msf'
header = 'test.header'
atom_style = 'full'
temp = 188 # K
num_voxels = [2,2,2]
x_bounds = []#[-1.4705252338620070e+02, 1.4705252338620070e+02]
y_bounds = []#[-1.4705252338620070e+02, 1.4705252338620070e+02]
z_bounds = []#[-1.4705252338620070e+02, 1.4705252338620070e+02]
diffusion_cutoff = 30

atoms, bonds, angles, dihedrals, impropers, box, adj_mat, extra_prop = parse_data_file(
    data_file, unwrap=True, atom_style=atom_style)
rxndata = parse_rxndf(rxndf)
masterspecies = parse_msf(msf)
data_header = parse_header(header)
rxnmatrix = get_rxnmatrix(rxndata, masterspecies)

BoltzmannConstant = 1.987204259e-3 # kcal / (K mol)
Epsilon = 1.0 # kcal / mol
R = 0.00198588  # kcal/mol/K

for r in rxndata.keys():
    rxndata[r]['rawrate'] = \
        rxndata[r]['A'][0] * \
        temp**rxndata[r]['b'][0] * \
        np.exp(-rxndata[r]['Ea'][0]/temp/R)

voxels = calc_voxels(
    num_voxels, box,
    xbounds=x_bounds,
    ybounds=y_bounds,
    zbounds=z_bounds)
voxelsmap, voxelsx, voxelsy, voxelsz = voxels2voxelsmap(voxels)
voxelID2idx = {k: idx for idx, k in enumerate(sorted(list(voxels.keys())))}
atomtypes2moltype = {}
for k, v in masterspecies.items():
    atomtypes2moltype[tuple(sorted([i[2] for i in v['Atoms']]))] = k

molecules = gen_molecules(atoms, atomtypes2moltype, voxelsmap, voxelsx, voxelsy, voxelsz)

diffusion_rate = {
        _: np.random.normal(5,5,size=(8,8))
        for _ in masterspecies.keys()
    }

rxnscaling = pd.DataFrame(
    index=[1],
    columns=sorted(rxndata.keys()),
    data={_: 1 for _ in rxndata.keys()}
)

In [46]:
diffusion_rate['A2'][0]

array([ 9.36074321, 10.80097831,  4.37190413, 10.42714821,  6.88825956,
        0.24473368,  4.58522153,  2.20880209])

In [56]:
rxn_type = 1
minimum_diffusion_rate = 10
rxn_count = 10
reactive_molecule_idxs_i = [idx for idx,molID in enumerate(molecules.ids) if molecules.mol_types[idx] == rxndata[rxn_type]['reactant_molecules'][0]]
reactive_molecule_idxs_j = [idx for idx,molID in enumerate(molecules.ids) if molecules.mol_types[idx] == rxndata[rxn_type]['reactant_molecules'][1]]
diffusion_rates_forward = np.array( [
                        [diffusion_rate[molecules.mol_types[molidx_i]][voxelID2idx[molecules.voxels[molidx_i]],voxelID2idx[molecules.voxels[molidx_j]]] for molidx_j in reactive_molecule_idxs_j]
                        for molidx_i in reactive_molecule_idxs_i],
    )
diffusion_rates_reverse = np.array( [
                        [diffusion_rate[molecules.mol_types[molidx_j]][voxelID2idx[molecules.voxels[molidx_j]],voxelID2idx[molecules.voxels[molidx_i]]] for molidx_i in reactive_molecule_idxs_i]
                        for molidx_j in reactive_molecule_idxs_j],
    )
diffusion_rates = np.maximum(diffusion_rates_reverse,diffusion_rates_forward)
cycle = rxnscaling.index[-1]
1 / (rxndata[rxn_type]['rawrate']*rxnscaling.loc[cycle,rxn_type])

upper_triangle_indices = np.triu_indices_from(diffusion_rates, k=1)

# Filter indices based on the minimum value
filtered_indices = [(i, j) for i, j in zip(*upper_triangle_indices) if diffusion_rates[i, j] > minimum_diffusion_rate]
print(diffusion_rates.size)
print(len(filtered_indices))
print(filtered_indices[30])
reactions = { reaction_index+1:
        [ rxn_type, [reactive_molecule_idxs_i[value[0]], reactive_molecule_idxs_j[value[1]]], diffusion_rates[value] ]
        for reaction_index,value in enumerate(filtered_indices)
}
print(reactions)

360000
33101
(0, 143)
{1: [1, [2, 4], 11.3018679817964], 2: [1, [2, 15], 11.3018679817964], 3: [1, [2, 24], 11.3018679817964], 4: [1, [2, 28], 10.302683089572856], 5: [1, [2, 38], 11.3018679817964], 6: [1, [2, 42], 11.3018679817964], 7: [1, [2, 45], 10.302683089572856], 8: [1, [2, 47], 11.3018679817964], 9: [1, [2, 55], 10.302683089572856], 10: [1, [2, 73], 11.3018679817964], 11: [1, [2, 75], 11.3018679817964], 12: [1, [2, 93], 11.3018679817964], 13: [1, [2, 94], 11.3018679817964], 14: [1, [2, 117], 11.3018679817964], 15: [1, [2, 118], 11.3018679817964], 16: [1, [2, 120], 11.3018679817964], 17: [1, [2, 123], 10.302683089572856], 18: [1, [2, 126], 10.302683089572856], 19: [1, [2, 128], 10.302683089572856], 20: [1, [2, 135], 10.302683089572856], 21: [1, [2, 144], 10.302683089572856], 22: [1, [2, 157], 11.3018679817964], 23: [1, [2, 162], 11.3018679817964], 24: [1, [2, 165], 10.302683089572856], 25: [1, [2, 168], 11.3018679817964], 26: [1, [2, 176], 10.302683089572856], 27: [1, [2, 193], 

179700


In [ ]:
def get_rxns_serial(molecules,voxelID2idx,diffusion_rates,rxnscaling,rxn_data,minimum_diffusion_rate):
    """Creates instance of class hybridmdmc.classes.ReactionList.

    Parameters
    ----------
   

    Returns
    -------
    rxns: instance of class HybridMDMC_Classes.ReactionList
    """
    # The total reaction rate is calculated as follows:
    # timeofdiffusion = 1 / diffusionrate
    #   diffusionrate is the rate of diffusion from voxel i to voxel j
    #   for bimolecular rxns, where the first reactant is in voxel i
    #   and the second is in voxel j
    # timeofrxn = 1 / (rawrxnrate * rxnscaling)
    # totalrxnrate = 1 / (timeofdiffusion + timeofrxn)
    #
    # It is assumed that if diffusion/reaction scaling is NOT
    # requested, all diffusion rates are np.inf/all reaction scalings
    # are 1.

    reactions, number_of_reactions = {},0
    cycle = rxnscaling.index[-1]

    for rxn_type in rxn_data.keys(): # Loop over every possible reaction in the rxn_data

        # Unimolecular reactions
        if len(rxn_data[rxn_type]['reactant_molecules']) == 1:
            reactive_molecule_idxs_i =  [idx for idx,molID in enumerate(molecules.ids) if molecules.mol_types[idx] in rxn_data[rxn_type]['reactant_molecules']]
            reactions.update({ reaction_index+1+number_of_reactions:
                    [
                        rxn_type, # reaction type
                        [molecules.ids[molecule_index]], # list of molecule ID
                        rxn_data[rxn_type]['rawrate']*rxnscaling.loc[cycle,rxn_type] # reactione event rate
                    ]
                    for reaction_index,molecule_index in enumerate(reactive_molecule_idxs_i)})
            continue

        # Bimolecular reactions
        elif len(rxn_data[rxn_type]['reactant_molecules']) == 2:
            reactive_molecule_idxs_i = [idx for idx,molID in enumerate(molecules.ids) if molecules.mol_types[idx] == rxndata[rxn_type]['reactant_molecules'][0]]
            reactive_molecule_idxs_j = [idx for idx,molID in enumerate(molecules.ids) if molecules.mol_types[idx] == rxndata[rxn_type]['reactant_molecules'][1]]
            diffusion_rates_forward = np.array( [
                                    [diffusion_rate[molecules.mol_types[molidx_i]][voxelID2idx[molecules.voxels[molidx_i]],voxelID2idx[molecules.voxels[molidx_j]]] for molidx_j in reactive_molecule_idxs_j]
                                    for molidx_i in reactive_molecule_idxs_i],
                )
            diffusion_rates_reverse = np.array( [
                                    [diffusion_rate[molecules.mol_types[molidx_j]][voxelID2idx[molecules.voxels[molidx_j]],voxelID2idx[molecules.voxels[molidx_i]]] for molidx_i in reactive_molecule_idxs_i]
                                    for molidx_j in reactive_molecule_idxs_j],
                )
            diffusion_rates = np.maximum(diffusion_rates_reverse,diffusion_rates_forward)
            upper_triangle_indices = np.triu_indices_from(diffusion_rates, k=1)
            filtered_indices = [(i, j) for i, j in zip(*upper_triangle_indices) if diffusion_rates[i, j] > minimum_diffusion_rate]
            times_of_diffusion = 1 / diffusion_rates
            times_of_rxn = 1 / (rxn_data[rxn_type]['rawrate']*rxnscaling.loc[cycle,rxn_type])
            reactions.update({ reaction_index+1+number_of_reactions:
                    [
                        rxn_type, # reaction type
                        [molecules.ids[reactive_molecule_idxs_i[value[0]]], molecules.ids[reactive_molecule_idxs_j[value[1]]]], # list of molecule IDs
                        1 / (times_of_diffusion[value] + times_of_rxn) # reaction event rate
                    ] for reaction_index,value in enumerate(filtered_indices)})
            number_of_reactions = len(reactions)
            continue

        # More than bimolecular rxn
        elif len(rxn_data[rxn_type]['reactant_molecules']) > 2:
            print('Error! hybridmdmc.functions.get_rxns does not currently support reactions between more than 2 molecules.')

    reactions = ReactionList(
        ids=sorted(list(reactions.keys())),
        rxn_types=[reactions[k][0] for k in sorted(list(reactions.keys())) ], # reaction types
        reactants=[reactions[k][1] for k in sorted(list(reactions.keys())) ], # lists of reactant molecule(s) ID(s)
        rates=[reactions[k][2] for k in sorted(list(reactions.keys())) ])     # rates

    return reactions

In [14]:
1/np.inf

0.0

In [12]:
600*600*8/1000/1000

2.88

In [17]:
np.array([
    [1,2,3],
    [4,5,6],
    [7,8,9],
])

array([[1, 2, 3],
       [4, 5, 6],
       [7, 8, 9]])